# Employee Evaluation Script

### Import Neccesary Packages and Files

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown
from ipynb.fs.full.MODULAR_DufflFeedbackAnalyzer import predictFeedbackType

## Change File Path to change week

In [78]:
school = "UofA-5"
dateRange = '11.27.23-12.3.23'
pathFolder = 'C:/Users/aidan/Documents/Python projects/Duffl/Eval CSVs/' + school + '/' + dateRange + '/'

In [79]:
#Get KPI spreadsheet -> rename "KPIs.csv"
#USE PERSONAL QUERY 'RAW FALSE POS'
KPIdf = pd.read_csv(pathFolder + 'KPIs.csv')

#Drop irrelevant columns
KPIdf = KPIdf.drop(['rank','store','position'], axis=1)

KPIdf.head(1)

,name,time_on_shift,num_orders,avg_delivery,avg_out_the_door,orh,percent_late,falses,verifications
0,Spencer Laumann,24.13636,150,10.2467,4.5849,6.21469,0.04,14,37


In [80]:
#Calculate false positive percent
KPIdf["false_pos"] = KPIdf["falses"]/KPIdf["verifications"]

#Drop raw false pos data for percent
KPIdf = KPIdf.drop(['falses','verifications'], axis=1)

#Check that false pos percent was added correctly
KPIdf.head(1)

,name,time_on_shift,num_orders,avg_delivery,avg_out_the_door,orh,percent_late,false_pos
0,Spencer Laumann,24.13636,150,10.2467,4.5849,6.21469,0.04,0.378378


In [81]:
#Get Customer Feedback spreadsheet -> rename "Feedback.csv"
Feedbackdf = pd.read_csv(pathFolder + 'Feedback.csv')

#Drop irrelevant columns
Feedbackdf = Feedbackdf.drop(['store','created_at_pst','phone','customer'], axis=1)

#Code sentiment type
# Positive = 1
# Neutral = 0
# Negative = -1
sentiment_dict={"positive":1, "neutral":0,"negative":-1}
Feedbackdf['sentiment'] = Feedbackdf['sentiment'].map(sentiment_dict)

#Predict who is responsible for feedback using imported function
predictFeedbackType(Feedbackdf)

#Show feedback with appended predictions
Feedbackdf.head(1)

C:\Users\aidan\Documents\Python projects\Duffl\MODULAR_DufflFeedbackAnalyzer.ipynb:117: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  "cell_type": "code",
C:\Users\aidan\Documents\Python projects\Duffl\MODULAR_DufflFeedbackAnalyzer.ipynb:115: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  },


,sentiment,feedback,racer,packer,Predictions
0,1,Everything bruh,Klein Powell,Klein Powell,R


In [82]:
#Function to clean training csvs
def clean_training_csv(training_name):
    Trainingdf = pd.read_csv(pathFolder + training_name + ".csv")
    Trainingdf = Trainingdf[Trainingdf['First name'] != 'Deleted user']
    Trainingdf = Trainingdf[Trainingdf['Course status'] == 'Completed']
    Trainingdf = Trainingdf.reset_index(drop = True)

    scores = []
    full_names = []
    for i in range (len(Trainingdf.index)):
        temp_name_string = Trainingdf.loc[i][0] + " " + Trainingdf.loc[i][1]
        full_names.append(temp_name_string)
        scores.append(Trainingdf.loc[i]["Overall score"])
        
    Trainingdf["Name"] = full_names
    Trainingdf["Score"] = scores
    Trainingdf = Trainingdf[["Name","Score"]]
    return Trainingdf

In [83]:
#Pick trainings you want to check
if school == 'UCLA-1':
    welcome_packet = clean_training_csv("Welcome Packet")
    welcome_packet.head(1)

In [84]:
if school == 'UCLA-1':
    post_orientation_packet = clean_training_csv("Post Orientation Packet")
    post_orientation_packet.head(1)

### Store Eval - Averages of the Week

In [85]:
#Sanity check to make sure the means makes sense with corresponding metric
KPIdf[['time_on_shift', 'num_orders', 'avg_delivery', 'avg_out_the_door', 'orh', 'percent_late', 'false_pos']].mean()

time_on_shift       12.077676
num_orders          49.156250
avg_delivery        10.824559
avg_out_the_door     4.702231
orh                  4.359056
percent_late         0.057178
false_pos            0.367918
dtype: float64

In [86]:
#Make new data frame for means
meandf = pd.DataFrame(columns=['Store'],
                        index=['Avg. Hrs Worked', 'Daily Avg. Orders', 
                               'Avg. Delivery Time', 'Avg. Out the Door', 
                               'ORH','Late %','False Pos %'], data=KPIdf[['time_on_shift', 'num_orders', 
                          'avg_delivery', 'avg_out_the_door', 
                          'orh', 'percent_late','false_pos']].mean().values)

#Benchmarks from exec, 'goal posts'
expectedHoursWorked = 12
expectedAvgDeliveryTime = 10
expectedAvgOuttheDoorTime = 4
expectedORH = 9
expectedLatePerc = 0.1
expectedFalsePos = 0.01
#CONSTS FOR ORDER TOTAL METRICS
weekly_orders = 3500
daily_orders = weekly_orders/7

#Append these benchmarks to meandf to visually compare to actual means
meandf['Benchmarks'] = [expectedHoursWorked, daily_orders, expectedAvgDeliveryTime, expectedAvgOuttheDoorTime, expectedORH, expectedLatePerc, expectedFalsePos]
meandf = meandf.reindex(columns=["Benchmarks","Store"])

#Total Orders special handling -> sum instead of mean
total_orders_row = pd.DataFrame(columns=['Benchmarks','Store'], index=['Total Orders'])
total_orders_row['Benchmarks']= weekly_orders
total_orders_row['Store']=KPIdf[['num_orders']].sum()[0]

#Add total orders metric
meandf = pd.concat([meandf, total_orders_row])

#Visually compare means to benchmarks from exec
display(meandf)

C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\2883489144.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  total_orders_row['Store']=KPIdf[['num_orders']].sum()[0]


,Benchmarks,Store
Avg. Hrs Worked,12.00,12.077676
Daily Avg. Orders,500.00,49.156250
Avg. Delivery Time,10.00,10.824559
Avg. Out the Door,4.00,4.702231
ORH,9.00,4.359056
Late %,0.10,0.057178
False Pos %,0.01,0.367918
Total Orders,3500.00,1573.000000


In [87]:
#Performance check on each metric -> grades store average compared benchmark
##row[0] is benchmark
##row[1] is actual
test_list = []

#Hrs worked
row = meandf.iloc[0].values
if row[1] < row[0]:
    test_list.append('N')
else:
    test_list.append('Y')

#Avg Orders (Daily)
row = meandf.iloc[1].values
if row[1] < row[0]:
    test_list.append('N')
else:
    test_list.append('Y')
    
#Avg Delivery Time
row = meandf.iloc[2].values
if row[1] > row[0]:
    test_list.append('N')
else:
    test_list.append('Y')
    
#Avg Out the Door
row = meandf.iloc[3].values
if row[1] > row[0]:
    test_list.append('N')
else:
    test_list.append('Y')

#ORH
row = meandf.iloc[4].values
if row[1] < row[0]:
    test_list.append('N')
else:
    test_list.append('Y')
    
#Late %
row = meandf.iloc[5].values
if row[1] > row[0]:
    test_list.append('N')
else:
    test_list.append('Y')
    
#False Pos %
row = meandf.iloc[6].values
if row[1] > row[0]:
    test_list.append('N')
else:
    test_list.append('Y')
    
#Total Orders
row = meandf.iloc[7].values
if row[1] < row[0]:
    test_list.append('N')
else:
    test_list.append('Y')

In [88]:
#Add test passes to meandf
meandf['Pass'] = test_list
meandf['Store'] = meandf['Store'].round(2)
display(meandf)

,Benchmarks,Store,Pass
Avg. Hrs Worked,12.00,12.08,Y
Daily Avg. Orders,500.00,49.16,N
Avg. Delivery Time,10.00,10.82,N
Avg. Out the Door,4.00,4.70,N
ORH,9.00,4.36,N
Late %,0.10,0.06,Y
False Pos %,0.01,0.37,N
Total Orders,3500.00,1573.00,N


## Compile Weekly Performance by Employee

In [89]:
#Make dataframe for employee comparison
storedf = meandf.copy()

#Drop store passes
storedf = storedf.drop(columns=['Pass'])

#Drop order metrics -> not applicable to racer in this format
storedf = storedf.drop(index=(['Total Orders','Daily Avg. Orders']))

#Reformat table for readability
storedf['Store'] = storedf['Store'].round(2)

display(storedf)

,Benchmarks,Store
Avg. Hrs Worked,12.00,12.08
Avg. Delivery Time,10.00,10.82
Avg. Out the Door,4.00,4.70
ORH,9.00,4.36
Late %,0.10,0.06
False Pos %,0.01,0.37


In [90]:
#Adds a list passed in as a parameter to the parameter df with column name of list_name parameter
def dfAddList(df, list_name, input_list):
    df[list_name] = input_list

In [91]:
#Takes a given employee and calculates their ZScore for each metric as compared to the store mean
def scoreCheck(df, employee_name):
    score_list = []
    
    #Generate standard deviations for each metric based on KPIdf
    stds = KPIdf[['time_on_shift', 
           'avg_delivery', 'avg_out_the_door', 
           'orh', 'percent_late', 'false_pos']].std().values
    
    #Drop store id and rank
    employee_metric = KPIdf.loc[KPIdf['name'] == employee_name].values
    employee_metric = np.delete(employee_metric, 0)
    employee_metric = np.delete(employee_metric, 1)
    
    #Drop the Benchmarks and Pass columns to get raw means
    means = meandf.drop(columns=['Benchmarks','Pass'])
    
    #Get means for all rows relevant to individual racer performance
    means = means.drop(index=(["Total Orders",'Daily Avg. Orders'])).values
    
    #Calculate the employee's ZScore for each metric
    for i in range (len(employee_metric)):
        currMetric = employee_metric[i]
        currStd = stds[i]
        currMean = means[i]
        
        score_list.append(((currMetric - currMean)/currStd)[0].round(2))
        
    #Append the ZScore column to the employee's eval df
    dfAddList(df, 'ZScore', score_list)

In [92]:
#Checks whether a give employee's performance passes the benchmarks
def testCheck(df, employee_name):
    #Performance check on each metric
    ##row[0] is benchmark
    ##row[1] is actual
    test_list = []
    
    #Numerical pass count
    pass_count = 0
    
    #Drop store id and rank
    employee_metric = KPIdf.loc[KPIdf['name'] == employee_name].values
    employee_metric = np.delete(employee_metric, 0)
    employee_metric = np.delete(employee_metric, 1)

    #Drop the store averages and pass column to isolate benchmarks
    benchmarks = meandf.drop(columns=['Store','Pass'])
    
    #Get benchmarks relevant to individual racer performance
    benchmarks = benchmarks.drop(index=(["Total Orders",'Daily Avg. Orders'])).values

    #Compare each metric to benchmark
    #Hrs worked
    if employee_metric[0] < benchmarks[0]:
        test_list.append('N')
    else:
        test_list.append('Y')
        pass_count += 1

    #Avg Delivery Time
    if employee_metric[1] > benchmarks[1]:
        test_list.append('N')
    else:
        test_list.append('Y')
        pass_count += 1

    #Avg Out the Door
    if employee_metric[2] > benchmarks[2]:
        test_list.append('N')
    else:
        test_list.append('Y')
        pass_count += 1

    #ORH
    if employee_metric[3] < benchmarks[3]:
        test_list.append('N')
    else:
        test_list.append('Y')
        pass_count += 1

    #Late %
    if employee_metric[4] > benchmarks[4]:
        test_list.append('N')
    else:
        test_list.append('Y')
        pass_count += 1

    #False Pos %
    if employee_metric[5] > benchmarks[5]:
        test_list.append('N')
    elif np.isnan(employee_metric[5]) == True:
        test_list.append('X')
    else:
        test_list.append('Y')
        pass_count += 1
    
     
    #Add test_list to the eval df
    dfAddList(df, 'Pass', test_list)
    
    return pass_count

In [93]:
#Gets all feedback relevant to a give employee based on the predicitons and employee's role
def getFeedback(employee_name):
    #Create a df to store the given employee's feedback
    individualFeedbackdf = pd.DataFrame(columns=['Positive Feedback','Negative Feedback'])
    
    #feedbackScore to store overall racer sentiment
    feedbackScore = 0
    
    #Store positive and negative reviews, repectively, per racer
    positiveList = []
    negativeList = []
    
    #Go through the entire feedback dataframe to find any relevant feedback
    for i in range (len(Feedbackdf.index)):
        #if POS and employee as Racer and Packer
        if (Feedbackdf.iloc[i][0] == 1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][3] == employee_name):
            positiveList.append('AS PACKER AND RACER: ' + Feedbackdf.iloc[i][1])
            negativeList.append('')
            feedbackScore += 1
        #if NEG/NEU and employee as Racer and Packer
        elif (Feedbackdf.iloc[i][0] == 0 or Feedbackdf.iloc[i][0] == -1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][3] == employee_name):
            negativeList.append('AS PACKER AND RACER: ' + Feedbackdf.iloc[i][1])
            feedbackScore -= 1
            positiveList.append('') 
        #if POS and employee as Racer and predcited Racer
        elif (Feedbackdf.iloc[i][0] == 1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][4] == 'R'):
            positiveList.append('AS RACER: ' + Feedbackdf.iloc[i][1])
            feedbackScore += 1
            negativeList.append('')
        #if POS and employee as Packer and predicted Packer
        elif (Feedbackdf.iloc[i][0] == 1) and (Feedbackdf.iloc[i][3] == employee_name) and (Feedbackdf.iloc[i][4] == 'P'):
            positiveList.append('AS PACKER: ' + Feedbackdf.iloc[i][1])
            feedbackScore += 1
            negativeList.append('')
        #NEG/NEU and employee as Racer and predicted Racer:
        elif (Feedbackdf.iloc[i][0] == 0 or Feedbackdf.iloc[i][0] == -1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][4] == 'R'):
            negativeList.append('AS RACER: ' + Feedbackdf.iloc[i][1])
            feedbackScore -= 1
            positiveList.append('') 
        #NEG/NEU and employee as Packer and predicted Packer:
        elif (Feedbackdf.iloc[i][0] == 0 or Feedbackdf.iloc[i][0] == -1) and (Feedbackdf.iloc[i][3] == employee_name) and (Feedbackdf.iloc[i][4] == 'P'):
            negativeList.append('AS PACKER: ' + Feedbackdf.iloc[i][1])
            feedbackScore -= 1
            positiveList.append('')
        #if POS and employee Inconclusive:
        elif (Feedbackdf.iloc[i][0] == 1) and (Feedbackdf.iloc[i][4] == 'I') and (Feedbackdf.iloc[i][2] == employee_name or Feedbackdf.iloc[i][3] == employee_name):
            positiveList.append('INCONCLUSIVE: ' + Feedbackdf.iloc[i][1])
            feedbackScore += 1
            negativeList.append('')
        #NEG/NEU and employee Inconclusive:
        elif (Feedbackdf.iloc[i][0] == 0 or Feedbackdf.iloc[i][0] == -1) and (Feedbackdf.iloc[i][4] == 'I') and (Feedbackdf.iloc[i][2] == employee_name or Feedbackdf.iloc[i][3] == employee_name):
            negativeList.append('INCONCLUSIVE: ' + Feedbackdf.iloc[i][1])
            feedbackScore -= 1
            positiveList.append('')

    #Store feedback in a dataframe
    individualFeedbackdf['Positive Feedback'] = positiveList
    individualFeedbackdf['Negative Feedback'] = negativeList

    #Show employee's feedback for jupyter eval display
    display(individualFeedbackdf)
    #Log employee's feedback in eval csv
    individualFeedbackdf.to_csv(pathFolder + dateRange + 'EmployeeEvals.csv', mode='a', header=True, index=False)

    #Remove columns to prime dataframe for next employee
    individualFeedbackdf = individualFeedbackdf.drop(columns=['Positive Feedback','Negative Feedback'])
    
    #Return a generalized pos or neg sentiment based on cumulative feedbackScore
    if feedbackScore > 0:
        return 1
    elif feedbackScore < 0:
        return -1
    else:
        return 0

In [94]:
#Check for training completion
def training_check(employee_name):
    #Create a df to store the given employee's feedback
    individualTrainingdf = pd.DataFrame(columns=['Training','Score'])
    
    trainings = ['Welcome Packet','Post Orientation Packet']
    scores = []
    #Welcome Packet
    if employee_name in welcome_packet['Name'].values:
        scores.append(str(welcome_packet.loc[welcome_packet['Name']==employee_name]['Score'].values[0]))
    else:
        scores.append("False")
    
    #Post Orientation Packet
    if employee_name in post_orientation_packet['Name'].values:
        scores.append(str(post_orientation_packet.loc[post_orientation_packet['Name']==employee_name]['Score'].values[0]))
    else:
        scores.append("False")
        
    individualTrainingdf['Training'] = trainings
    individualTrainingdf['Score'] = scores   

    #Show employee's feedback for jupyter eval display
    display(individualTrainingdf)
    #Log employee's feedback in eval csv
    individualTrainingdf.to_csv(pathFolder + dateRange + 'EmployeeEvals.csv', mode='a', header=True, index=False)

In [95]:
#Formatting for eval df
def make_pretty(styler):
    #Round to 2 decimal places for readability
    styler.format(precision = 2)
    #Positively coded ZScore conditional formatting
    styler.background_gradient(axis=None, subset=(['Avg. Delivery Time','Avg. Out the Door','Late %','False Pos %'], ['ZScore']), vmin=-2.5, vmax=2.5, cmap="RdYlGn_r")
    #Negatively coded ZScore conditional formatting
    styler.background_gradient(axis=None, subset=(['Avg. Hrs Worked','ORH'], ['ZScore']), vmin=-2.5, vmax=2.5, cmap="RdYlGn")
    return styler

In [96]:
#Generate eval for each racer in the KPI for given date range
for i in range (len(KPIdf.index)):
    #Get row from KPI sheet
    racerdf = KPIdf.iloc[i]

    #Get employee name then drop it from row
    employee_name = racerdf[0]
    racerdf = racerdf.drop((['name','num_orders']))

    #Display title for jupyter viewing of employee evals
    display_markdown('''## ''' + employee_name + ''''s Eval:''', raw=True)

    #Check whether racer's metrics passed benchmarks
    num_test_score = testCheck(storedf, employee_name)
    
    #Calcualte ZScore for racer's metrics
    scoreCheck(storedf, employee_name)
    
    #Show calc results in table
    storedf[employee_name] = racerdf.values  
        
    #If there were no verifications give an 'X' rating for Pass column
    if storedf[employee_name][5] == -1:
        storedf[employee_name][5] = 'X'
        
    #Create a copy of df to style -> styled can't be stored in export CSV
    storedfStyled = storedf.copy()
    storedfStyled = storedfStyled.style.pipe(make_pretty)
    
    #Display formatted CSV for jupyter eval viewing
    display(storedfStyled)

    #Log evals in export CSV
    storedf.to_csv(pathFolder + dateRange + 'EmployeeEvals.csv', mode='a', header=True, index=True)

    #Get any relevant feedback from this week
    display_markdown('''## Feedback:''', raw=True)
    overall_feedback_score = getFeedback(employee_name)

    overall_score = overall_feedback_score + num_test_score
    
    #Display trainings done
    if school == 'UCLA-1':
        display_markdown('''## Trainings:''', raw=True)
        training_check(employee_name)
    
    #Display racer score for jupyter viewing of employee evals
    display_markdown('''## ''' + employee_name + ''''s Score: ''' + str(overall_score) + '''/6''', raw=True)
    
    display_markdown('''### ------------------------------------------------------------------------------------------''', raw=True)
    
    #Spacing for formatting in csv
    spacingdf = pd.DataFrame(columns=[' '],index=[' ',' ',' '])
    s = pd.Series([' '],index=[' '])
    
    #Formatting -> extra three empty rows to space out in exported CSV
    #spacingdf = spacingdf.append(s,ignore_index=True)
    #spacingdf = spacingdf.append(s,ignore_index=True)
    #spacingdf = spacingdf.append(s,ignore_index=True)
    
    spacingdf.to_csv(pathFolder + dateRange + 'EmployeeEvals.csv', mode='a', header=True, index=False)
    
    #End of loop -> prime df for next employee
    storedf = storedf.drop(columns=[employee_name])

C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\2927901798.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  employee_name = racerdf[0]


## Spencer Laumann's Eval:

C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\2927901798.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if storedf[employee_name][5] == -1:


,Benchmarks,Store,Pass,ZScore,Spencer Laumann
Avg. Hrs Worked,12.00,12.08,Y,1.94,24.14
Avg. Delivery Time,10.00,10.82,N,-0.26,10.25
Avg. Out the Door,4.00,4.70,N,-0.09,4.58
ORH,9.00,4.36,N,1.10,6.21
Late %,0.10,0.06,Y,-0.27,0.04
False Pos %,0.01,0.37,N,0.06,0.38


## Feedback:

...

## Feedback:

C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\158198044.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (Feedbackdf.iloc[i][0] == 1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][3] == employee_name):
C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\158198044.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif (Feedbackdf.iloc[i][0] == 0 or Feedbackdf.iloc[i][0] == -1) and (Feedbackdf.iloc[i][2] == employee_name) and (Feedbackdf.iloc[i][3] == employee_name):
C:\Users\aidan\AppData\Local\Temp\ipykernel_27120\158198044.py:26: FutureWarning: Series.__getitem__ treating keys as p

,Positive Feedback,Negative Feedback


## Davis Ong's Score: 0/6

### ------------------------------------------------------------------------------------------